In [1]:
import os
import tensorflow as tf
from PIL import UnidentifiedImageError
from tensorflow.keras.preprocessing.image import (load_img, img_to_array, ImageDataGenerator)
from tensorflow.keras.applications.vgg16 import (preprocess_input, decode_predictions, VGG16)
from tensorflow.keras.layers import (Flatten, Dense, Dropout, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD, Adam
import keras
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier

2024-05-17 11:22:28.852832: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 11:22:28.856621: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 11:22:28.908530: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 11:22:29.817726: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


KeyboardInterrupt: 

In [ ]:

def parser():
    """
    The user can specify which optimizer to use, whether to perform GridSearch to tune the hyperparameters, and
    to implement batch normalization and/or data augmentation.
    The function will then parse command-line arguments and make them lower case.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("--optimizer",
                        "-o",
                        required = True,
                        choices = ["adam", "sgd"],
                        help = "Choose optimizer") 
    parser.add_argument("--GridSearch",
                        "-gs",
                        required = True,
                        choices = ["yes", "no"],
                        help = "Perform GridSearch (yes or no)")
    parser.add_argument("--BatchNorm",
                        "-bn",
                        required = True,
                        choices = ["yes", "no"],
                        help = "Perform batch normalization (yes or no)")   
    parser.add_argument("--DatAug",
                        "-da",
                        required = True,
                        choices = ["yes", "no"],
                        help = "Perform data augmentation (yes or no)")                 
    args = parser.parse_args()
    args.optimizer = args.optimizer.lower()
    args.GridSearch = args.GridSearch.lower()
    args.BatchNorm = args.BatchNorm.lower()
    args.DatAug = args.DatAug.lower()
    return args

In [14]:


def load_images(folder_path):
    """
    Loads the data from the specified folder path, generates labels for each image, 
    and preprocesses them for model input.
    The dataset contains certain files, i.e., Thumbs.db, could not be loaded and
    returned the error 'UnidentifiedImageError'. These will simplt be ignored.
    """
    list_of_images = [] 
    list_of_labels = []
    
    for subfolder in sorted(os.listdir(folder_path)):
        subfolder_path  = os.path.join(folder_path, subfolder)
        
        for file in os.listdir(subfolder_path):
            individual_filepath = os.path.join(subfolder_path, file)
            
            try:
                image = load_img(individual_filepath, target_size = (224, 224))
                image = img_to_array(image)
                list_of_images.append(image)

                label = subfolder_path.split("/")[-1]
                list_of_labels.append(label)

            except (UnidentifiedImageError):
                print(f"Skipping {individual_filepath}")
        
    array_of_images = np.array(list_of_images)
    X = preprocess_input(array_of_images)
    y = list_of_labels
    
    return X, y


def data_split(X, y):
    """
    Splits the data into training and testing sets (80:20 split) by stratifing y. Normalizes X
    by simply dividing by the maximum possible value and performs label binarization on y.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 123)
    X_train = X_train.astype("float32") / 255.
    X_test = X_test.astype("float32") / 255.
    lb = LabelBinarizer()
    y_train = lb.fit_transform(y_train)
    y_test = lb.fit_transform(y_test) 

    return X_train, X_test, y_train, y_test


def define_model(BatchNorm):
    """
    Defines the model architecture. First, the VGG16 model is loaded from TensorFlow without the classification
    layers. The convolutional layers are marked as not trainable to retain their pretrained weights. The
    user specifies whether the model should be defined with or without batch normalization. Subsequently, a
    new fully connected layer with ReLU activation is added followed by an output layer with softmax
    activation for multi-class classification.
    """
    model = VGG16(include_top = False, pooling = 'avg', input_shape = (224, 224, 3))

    for layer in model.layers:
        layer.trainable = False

    if BatchNorm == "no":
        flat1 = Flatten()(model.layers[-1].output)
        class1 = Dense(128, activation = 'relu')(flat1)
        output = Dense(10, activation = 'softmax')(class1)

    elif BatchNorm == "yes":
        flat1 = Flatten()(model.layers[-1].output)
        bn = BatchNormalization()(flat1)
        class1 = Dense(128, activation='relu')(bn)
        output = Dense(10, activation='softmax')(class1)
    
    model = Model(inputs = model.inputs, outputs = output)

    return model




def sklearn_object(model):
    """
    Convert the model from a KerasClassifier to an object, that can be used in a scikit-learn pipeline.
    """
    #return KerasClassifier(model = model, verbose = 0)
    return KerasClassifier(build_fn = model, verbose = 0)


def data_generator():
    """
    The function creates an image data generator with ImageDataGenerator from TensorFlow.
    The implemented data augmentation involves settings as horizontal flipping and rotation.
    A validation split is set to 10%.
    """
    datagen = ImageDataGenerator(horizontal_flip = True, 
                                rotation_range = 90,
                                validation_split = 0.1)
    return datagen


def fit_model(model, X_train, y_train, DatAug, batchsize = 32, epochs = 10):
#def fit_model(model, X_train, y_train, DatAug, batchsize = batchsize, epochs = epochs):
    """
    The function fits the defined and compiled model to the training data.

    If the parameters have not been gridsearch to obtain the best parameters, the model will use a batch
    size of 32 and 10 epochs as default. 

    The user specifies whether to implement data augmentation...
    Fits the compiled model to the training data with or without data augmentation and returns the training history.

    early stopping implemented to minimise loss and avoid overfitting. Monitors val_loss
    After 3 epochs with no improvement, the training will be stopped
    
    When the model is fitted the function returns the fitted model, H.
    """

    if DatAug == "no":
        H = model.fit(X_train, y_train, 
                    validation_split = 0.1,
                    batch_size = batchsize,
                    epochs = epochs,
                    callbacks = EarlyStopping(monitor = 'val_loss', patience = 3, mode='min'))
    
    elif DatAug == "yes":
        DatAug = data_generator()
        datagen.fit(X_train)
        H = model.fit(datagen.flow(X_train, y_train, batch_size = batchsize),
                                    validation_data = datagen.flow(X_train, y_train, 
                                                                    batch_size = batchsize,
                                                                    subset = "validation"),
                                                                    epochs = epochs,
                                                                    callbacks = EarlyStopping(monitor = 'val_loss', patience = 3, mode='min')) 
    return H, batchsize, epochs


def plot_history(H, epochs, model_param, Optimizer, outpath):
    """
    Plots the training and validation loss and accuracy curves and saves the plot to a specified outpath.
    """

    plt.figure(figsize = (12,6))
    plt.suptitle(f"Training and validation curves with {model_param} paramters and {Optimizer} optimizer", fontsize = 8)

    plt.subplot(1,2,1)
    plt.plot(np.arange(0, epochs), H.history["loss"], label = "training loss")
    plt.plot(np.arange(0, epochs), H.history["val_loss"], label = "validation loss", linestyle = ":")
    plt.title("Loss curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.tight_layout()
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(np.arange(0, epochs), H.history["accuracy"], label = "training accuracy")
    plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label = "validation accuracy", linestyle = ":")
    plt.title("Accuracy curve")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.tight_layout()
    plt.legend()
    plt.savefig(outpath)
    plt.show()


def evaluate(model, X_test, y_test, H, batchsize, epochs, BatchNorm, DatAug, Optimizer):
    """
    Evaluates the model on the test data, generates classification reports, and saves the results.
    """
    label_names = ["ADVE", "Email", "Form", "Letter", "Memo", "News", "Note", "Report", "Resume", "Scientific"]
    predictions = model.predict(X_test, batch_size = batchsize)
    #y_preds = model.predict(X_test)
    classifier_metrics = classification_report(y_test.argmax(axis = 1),
                                               predictions.argmax(axis = 1),
                                               target_names = label_names)

    if BatchNorm == "yes":
        if DatAug == "yes": 
            model_param = "BatchNorm_DatAug"
        elif DatAug == "no":
            model_param = "BatchNorm"
    elif BatchNorm == "no": 
        if DatAug == "yes":
            model_param = "DatAug"
        elif DatAug == "no": 
            model_param = "baseline"

    filepath_metrics = open(f'out/{model_param}_metrics_{Optimizer}.txt', 'w')
    filepath_metrics.write(classifier_metrics)
    filepath_metrics.close()

    plot_history(H, epochs, model_param, Optimizer, f"out/{model_param}_losscurve_{Optimizer}.png")

    return print("Results have been saved to the out folder")


def compile_model(model, Optimizer):
    """
    The function compiles the model with the specified optimizer and respective learning rate
    """
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 0.01, 
                                                                decay_steps = 10000, decay_rate = 0.9)
    if Optimizer == "adam":
        optimizer = Adam(learning_rate = lr_schedule)
    if Optimizer == "sgd":
        optimizer = SGD(learning_rate = lr_schedule)
    model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

    return model 

def grid_search(model, X_train, y_train): #Optimizer
    """
    The function initially converts the model from a KerasClassifier to an object, that can be used in a
    scikit-learn pipeline. Afterwards, it performs GridSearch to find the best hyperparameters for the model.
    The best parameters will be returned.
    """
    #lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 0.01, 
    #                                                            decay_steps = 10000, decay_rate = 0.9)
    #if Optimizer == "adam":
    #    optimizer = Adam(learning_rate = lr_schedule)
    #if Optimizer == "sgd":
    #    optimizer = SGD(learning_rate = lr_schedule)
    #model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    model = KerasClassifier(model = model, verbose = 1)

    param_grid = {'epochs': [2],'batch_size': [16, 32]}
    #param_grid = {'epochs': [10, 15, 20],
    #            'batch_size': [16, 32, 64]}

    grid_search = GridSearchCV(estimator = model, param_grid = param_grid, cv = 2, n_jobs = -1, scoring = 'accuracy', verbose = 1)

    grid_result = grid_search.fit(X_train, y_train)

    print(f'Best Accuracy for {grid_result.best_score_} using the parameters {grid_result.best_params_}')

    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print(f' mean = {mean:.4}, std = {stdev:.4} using {param}')

    best_estimator = grid_result.best_estimator_
    batchsize, epochs = list(grid_result.best_params_.values())
    return best_estimator, batchsize, epochs


In [ ]:

def main():
    
    args = parser()

    folder_path = os.path.join("../../../../cds-vis-data/Tobacco3482") # ("in/Tobacco3482")

    X, y = load_images(folder_path)
    X_train, X_test, y_train, y_test = data_split(X, y)
    
    # define model - BatchNorm yes/no
    model = define_model(args.BatchNorm)

    # compile - choose optimizer
    model = compile_model(model, args.Optimizer)

    # grid search + fit
    if args.GridSearch == 'yes':
        model, batchsize, epochs = grid_search(model, X_train, y_train)
        H = fit_model(model, X_train, y_train, args.DatAug, batchsize = batchsize, epochs = epochs)
    else:
        H, batchsize, epochs = fit_model(model, X_train, y_train, args.DatAug, batchsize = 32, epochs = 10)
  
    # evaluate
    evaluate(model, X_test, y_test, H, batchsize, epochs, args.BatchNorm, args.DatAug, args.Optimizer)

if __name__ == "__main__":
    main()

In [15]:
folder_path = os.path.join("../../../../../cds-vis-data/Tobacco3482") # ("in/Tobacco3482")

X, y = load_images(folder_path)
X_train, X_test, y_train, y_test = data_split(X, y)

Skipping ../../../../../cds-vis-data/Tobacco3482/ADVE/Thumbs.db
Skipping ../../../../../cds-vis-data/Tobacco3482/Email/Thumbs.db
Skipping ../../../../../cds-vis-data/Tobacco3482/Form/Thumbs.db
Skipping ../../../../../cds-vis-data/Tobacco3482/Letter/Thumbs.db
Skipping ../../../../../cds-vis-data/Tobacco3482/Memo/Thumbs.db
Skipping ../../../../../cds-vis-data/Tobacco3482/News/Thumbs.db
Skipping ../../../../../cds-vis-data/Tobacco3482/Note/Thumbs.db
Skipping ../../../../../cds-vis-data/Tobacco3482/Report/Thumbs.db
Skipping ../../../../../cds-vis-data/Tobacco3482/Resume/Thumbs.db
Skipping ../../../../../cds-vis-data/Tobacco3482/Scientific/Thumbs.db


In [16]:
model = define_model("no")

In [17]:
model = compile_model(model, "adam")

In [18]:
model, batchsize, epochs = grid_search(model, X_train, y_train)

/home/ucloud/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 9 variables whereas the saved optimizer has 1 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Fitting 2 folds for each of 2 candidates, totalling 4 fits


2024-05-17 10:53:10.706779: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 10:53:10.710761: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 10:53:10.759554: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 10:53:10.993007: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 10:53:10.997214: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 10:53:11.050310: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary 

Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
44/44 ━━━━━━━━━━━━━━━━━━━━ 653s 15s/step - accuracy: 0.2944 - loss: 2.0118
Epoch 2/2
44/44 ━━━━━━━━━━━━━━━━━━━━ 657s 15s/step - accuracy: 0.3145 - loss: 1.9759
Epoch 2/2
87/87 ━━━━━━━━━━━━━━━━━━━━ 827s 10s/step - accuracy: 0.3298 - loss: 1.9328
Epoch 2/2
88/88 ━━━━━━━━━━━━━━━━━━━━ 827s 9s/step - accuracy: 0.3544 - loss: 1.9109
Epoch 2/2
44/44 ━━━━━━━━━━━━━━━━━━━━ 636s 14s/step - accuracy: 0.5620 - loss: 1.3032
44/44 ━━━━━━━━━━━━━━━━━━━━ 644s 15s/step - accuracy: 0.5403 - loss: 1.3208
87/87 ━━━━━━━━━━━━━━━━━━━━ 827s 10s/step - accuracy: 0.5677 - loss: 1.2415
88/88 ━━━━━━━━━━━━━━━━━━━━ 851s 9s/step - accuracy: 0.5235 - loss: 1.3015
 9/88 ━━━━━━━━━━━━━━━━━━━━ 10:36 8s/step

10/88 ━━━━━━━━━━━━━━━━━━━━ 9:37 7s/step 